# YouTube
Tem como objetivo baixar videos no YouTube.

## Dependencias SO
- [FFmpeg](https://ffmpeg.org/) 

## Setup

In [1]:
from pytube import YouTube as PyTube, StreamQuery
from pathlib import Path
import pandas as pd
from IPython.display import clear_output
from functools import cache
import shlex
import ffmpeg

YOUTUBE_PATH = Path("./files/youtube")
YOUTUBE_PATH.mkdir(parents=True, exist_ok=True)


class YouTube:
    DOWNLOAD_BLOCKS = 20

    def __init__(self, url: str):
        self._yt = PyTube(url=url, on_progress_callback=self._progress)
        self.url = url
        self.autor = self._yt.author
        self.titulo = self._yt.title
        self._filesize: int = 0
        self._last_itag: int = None

    def __str__(self) -> str:
        return f"<Youtube autor={shlex.quote(self.autor)} titulo={shlex.quote(self.titulo)} url={shlex.quote(self.url)}>"

    def __repr__(self) -> str:
        return str(self)

    @property
    def _streams(self) -> StreamQuery:
        @cache
        def wrapper():
            return self._yt.streams
        return wrapper()

    def _progress(self, chunk, file_handle: bytes, bytes_remaining: int):
        percent = ((self._filesize - bytes_remaining) * 100) / self._filesize
        block = int((percent * self.DOWNLOAD_BLOCKS) / 100)
        clear_output(wait=True)
        print("Progresso: [{0}] {1:.1f}% | itag: {2}".format("#" * block + "-" * (20 - block), percent, self._last_itag))
    
    def _baixar_stream(self, itag: int, output_path: Path = YOUTUBE_PATH, filename_prefix: str = None) -> str:
        output_path.mkdir(exist_ok=True, parents=True)
        stream = self._streams.get_by_itag(itag)
        self._filesize += stream.filesize
        self._last_itag = itag
        return stream.download(output_path=str(output_path), filename_prefix=filename_prefix)

    def resolucoes(self, unico_arquivo: bool = False) -> pd.DataFrame:
        if unico_arquivo:
            resolutions = self._streams.filter(progressive=True)
        else:
            resolutions = self._streams
        df = pd.DataFrame([res.__dict__ for res in resolutions])
        return df

    def baixar_arquivo(self, itag: int) -> str:
        self._filesize = 0
        self._last_itag = None
        return self._baixar_stream(itag=itag)

    def baixar_arquivo_dash(self, video_itag: int, audio_itag: int) -> str:
        self._filesize = 0
        self._last_itag = None
        temp_path = YOUTUBE_PATH.joinpath("temp")
        faixas: list[str] = []
        for itag in (video_itag, audio_itag):
            faixas.append(
                self._baixar_stream(itag=itag, output_path=temp_path, filename_prefix=f"{itag!s} ")
            )
        primeiro_arquivo = Path(faixas[0])
        nome, extensao = primeiro_arquivo.name, primeiro_arquivo.suffix
        YOUTUBE_PATH.mkdir(exist_ok=True, parents=True)
        arquivo_saida = YOUTUBE_PATH.joinpath(nome.replace(primeiro_arquivo.suffix, ".mp4").replace(str(video_itag), "").strip())
        ffmpegs = [ffmpeg.input(p) for p in faixas]
        ffmpeg.concat(*ffmpegs, v=1, a=1).output(str(arquivo_saida)).run()
        [Path(p).unlink(missing_ok=True) for p in faixas]
        return str(arquivo_saida.resolve())


## Execução
Executar os 3 passos abaixo na sequencia:

### 1. Digitar URL do video

In [2]:
youtube_url = "https://www.youtube.com/watch?v=BhF6mkQtObw"
yt = YouTube(url=youtube_url)
yt

<Youtube autor='Copa Studio' titulo='Irmão do Jorel - Vovó Me Leva pra Aula - Completo (Oficial)' url='https://www.youtube.com/watch?v=BhF6mkQtObw'>

### 2. Listar Resoluções

In [8]:
# Somente campos
", ".join(list(yt.resolucoes()))

'_monostate, url, itag, mime_type, codecs, type, subtype, video_codec, audio_codec, is_otf, bitrate, _filesize, _filesize_kb, _filesize_mb, _filesize_gb, is_dash, abr, fps, resolution, is_3d, is_hdr, is_live'

In [3]:
pd.set_option("max_colwidth", None)
# yt.resolucoes(unico_arquivo=True)[["itag", "mime_type", "video_codec", "audio_codec", "fps", "resolution", "abr"]]
yt.resolucoes()[["itag", "mime_type", "video_codec", "audio_codec", "fps", "resolution", "abr", "is_dash"]]

,itag,mime_type,video_codec,audio_codec,fps,resolution,abr,is_dash
0,17,video/3gpp,mp4v.20.3,mp4a.40.2,12.0,144p,24kbps,False
1,18,video/mp4,avc1.42001E,mp4a.40.2,24.0,360p,96kbps,False
2,22,video/mp4,avc1.64001F,mp4a.40.2,24.0,720p,192kbps,False
3,137,video/mp4,avc1.640028,None,24.0,1080p,None,True
4,248,video/webm,vp9,None,24.0,1080p,None,True
5,399,video/mp4,av01.0.08M.08,None,24.0,1080p,None,True
6,136,video/mp4,avc1.4d401f,None,24.0,720p,None,True
7,247,video/webm,vp9,None,24.0,720p,None,True
8,398,video/mp4,av01.0.05M.08,None,24.0,720p,None,True
9,135,video/mp4,avc1.4d401e,None,24.0,480p,None,True


### 3. Baixar Video

In [5]:
yt.baixar_arquivo(itag=22)

Progresso: [####################] 100.0% | itag: 22


'/home/douglaspands/workspace/vscode-notebook/files/youtube/Irmão do Jorel - Vovó Me Leva pra Aula - Completo (Oficial).mp4'

In [4]:
# DASH: é necessario passar 2 itags: somente video e somente audio.
# Essa forma contem melhor qualidade. (De preferencia para formatos de video e audio MP4)
yt.baixar_arquivo_dash(video_itag=137, audio_itag=140)

Progresso: [####################] 100.0% | itag: 140


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

'/home/douglaspands/workspace/vscode-notebook/files/youtube/Irmão do Jorel - Vovó Me Leva pra Aula - Completo (Oficial).mp4'